In [ ]:

import csv
import argparse
import json
import pandas as pd

# import simplejson as json
annotations_file = "../open_images/train-annotations-bbox.csv"
descriptions_file="../open_images/class-descriptions-boxable.csv"

#download from google big-query, through web or API
image_map_file="marine_objs.json"
image_root = "../open_images/aerial_marine_objs"
destination_root = "."

with_small_image = True
image_key_name = 'thumbnail_300k_url'
if not with_small_image:
    image_key_name = original_url

    
#open image dataset labels that you want    
labels = ['/m/0199g', '/m/019jd', '/m/019w40', '/m/04dr76w', '/m/05gqfk',
       '/m/083wq', '/m/0h9mv','']

image_lookup = {}

In [ ]:
labeled_classes = pd.read_csv(descriptions_file,names=['label','description'])
target_labels = labeled_classes[labeled_classes['description'].isin(["Surfboard","Boat","Plastic bag", "Tire","Bottle","Bicycle","Wheel"])]
labels = target_labels['label'].values
descriptions = target_labels['description'].values
target_labels


In [ ]:
#make sure it's parsable
infile = open(image_map_file, 'r')
firstLine = infile.readline()
d = json.loads(firstLine)
iid = d['image_id']
# '{"image_id":"3894576d8d7f9cd1","original_url":"https://farm2.staticflickr.com/3513/3974970554_11da5a46e6_o.jpg","confidence":"1.0"}\n'

In [ ]:
import os
#this would automatically dedup
def bigquery_result_to_image_lookup(image_map_file):
    image_lookup = {}
    
    with open(image_map_file, 'r',encoding='utf8') as data_file:
        json_data = data_file.read()
    
    json_str_array = json_data.split("\n")
    jsons = []
    
    for st in json_str_array:
        try:
            jsons.append(json.loads(st))
        except:
            print("failed at {}".format(len(jsons)))
#             break
    for j in jsons:
        j['filename'] = os.path.basename(j[image_key_name])    
        image_lookup[j["image_id"]] = j
    return image_lookup


In [ ]:
image_lookup = bigquery_result_to_image_lookup(image_map_file)  

In [ ]:
len(image_lookup)

In [ ]:
def dedupe(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def format_annotations(annotation_path,description_table, labels, small_image=True, debug=False):
    annotations = []
    ids = []
    trainable_classes = labels
        
    with open(annotation_path) as annofile:
        reader = csv.reader(annofile)
        next(reader, None)  # skip the headers
        for row in reader:
            if row[0]=="ImageID":
                continue
            annotation = {'id': row[0], 'label': row[2], 'confidence': row[3], 'xratio0': row[4],
                          'xratio1': row[5], 'yratio0': row[6], 'yratio1': row[7]}
            if annotation['label'] in trainable_classes and annotation['id'] in image_lookup:                
                try:
                    image_filename = image_lookup[annotation['id']]['filename']
                    image_path = "{}/{}".format(image_root, image_filename)
                    pil_im = Image.open(image_path, 'r')
                    annotation['width'] = pil_im.width
                    annotation['height'] = pil_im.height
                    #keras retinanet only accepts absolute positions
                    annotation['x1'] = round(pil_im.width * float(annotation['xratio0']))
                    annotation['x2'] = round(pil_im.width * float(annotation['xratio1']))
                    annotation['y1'] = round(pil_im.height * float(annotation['yratio0']))
                    annotation['y2'] = round(pil_im.height * float(annotation['yratio1']))
                    annotation['class']= description_table[row[2]]
                    annotation['file_path'] = image_path

                    annotations.append(annotation)
                    ids.append(row[0])
                except:
                    if debug:
                        print("{} may not be available anymore".format(annotation['id']))
            
            
    ids = dedupe(ids)
    return annotations, ids

annotations, ids = format_annotations(annotations_file,description_table,labels, with_small_image)



In [ ]:
#make sure JSON looks ok
image_lookup[iid]

In [ ]:
image_filename = image_lookup[iid]['filename']
# ubuntu_image_root = "/home/ubuntu/object_detection/jelly_fishes"
image_path = "{}/{}".format(image_root, image_filename)
print(image_path)

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline
pil_im = Image.open(image_path, 'r')
imshow(np.asarray(pil_im))

In [ ]:
print(pil_im.width)
print(pil_im.height)

In [ ]:
annotations

### Export to csv 
- that you can use this with https://github.com/fizyr/keras-retinanet or some other object detection framework



In [ ]:
import csv

keras_annotations=[]
for a in annotations:
    keras_annotations.append({ keras_key: a[keras_key] for keras_key in ['file_path', 'x1', 'y1', 'x2', 'y2', 'class'] })

    
with open("{}/annotations.csv".format(destination_root), 'w') as f: 
    w = csv.DictWriter(f, keras_annotations[0].keys())
    w.writerows(keras_annotations)

In [ ]:
def process_images(saved_images_path, resized_images_path, points):
    cleaned_points = []
    for point in tqdm(points, desc="checking if images are valid from label index"):
            try:
                stored_path = os.path.join(saved_images_path, point['id'] + '.jpg')
                im = Image.open(stored_path)
                im.verify()
                # Now that the image is verified,
                # lets rescale it and overwrite.
                im.thumbnail((256, 256))
                if resized_images_path:
                    resized_path = os.path.join(resized_images_path, point['id'] + '.jpg')
                    im.save(resized_path, 'JPG')
                else:
                    os.remove(stored_path)
                    im.save(stored_path, 'JPG')
                cleaned_points.append(point)
            except:
                pass
    return cleaned_points

